# Multiclass Classification with Convolutional Neural Networks
Version 1.0

(C) 2020 - Umberto Michelucci, Michela Sperti  
(C) 2023 - Alessandro Motta

This notebook is part of the book _Applied Deep Learning: a case based approach, **2nd edition**_ from APRESS by [U. Michelucci](mailto:umberto.michelucci@toelt.ai) and [M. Sperti](mailto:michela.sperti@toelt.ai).

The purpose of this notebook is to give a practical example (with a dataset taken from the real world) of a multiclass classification problem solved by means of a Convolutional Neural Network (CNN) architecture. It may be instructive to compare the results of this notebook with the ones of *Multiclass_classification_with_fully_connected_networks.ipynb* notebook, which solves the same problem, but with a simpler architecture.

## Notebook Learning Goals

At the end of the notebook you are going to know how to implement yourself a CNN architecture in Keras. Moreover, you will be able to to apply it to other image recognition problems.

## Dataset Overview

**Context**

Fashion-MNIST is a dataset of Zalando's article images (consisting of a training set of 60000 examples and a test set of 10000 examples). Each example is a 28x28 grayscale image, associated with a label from 10 classes. Zalando intends Fashion-MNIST to serve as a direct drop-in replacement for the original MNIST dataset for benchmarking machine learning algorithms. It shares the same image size and structure of training and testing splits.

The original MNIST dataset contains a lot of handwritten digits. Members of the AI/ML/Data Science community love this dataset and use it as a benchmark to validate their algorithms. In fact, MNIST is often the first dataset researchers try. "If it doesn't work on MNIST, it won't work at all", they said. "Well, if it does work on MNIST, it may still fail on others."
Zalando seeks to replace the original MNIST dataset

**Content**


Each image is 28 pixels in height and 28 pixels in width, for a total of 784 pixels in total. Each pixel has a single pixel-value associated with it, indicating the lightness or darkness of that pixel, with higher numbers meaning darker. This pixel-value is an integer between 0 and 255. The training and test data sets have 785 columns. The first column consists of the class labels (see above), and represents the article of clothing. The rest of the columns contain the pixel-values of the associated image.

To locate a pixel on the image, suppose that we have decomposed $x$ as $x = 28i + j$, where $i$ and $j$ are integers between 0 and 27. The pixel is located on row $i$ and column $j$ of a 28x28 matrix.
For example, pixel31 indicates the pixel that is in the fourth column from the left, and the second row from the top.

Each row of the dataset is a separate image. Column 1 is the class label.
Remaining columns are pixel numbers (784 total). Each value is the darkness of the pixel (1 to 255).

**Labels**

Each training and test example is assigned to one of the following labels:
- 0 T-shirt/top
- 1 Trouser
- 2 Pullover
- 3 Dress
- 4 Coat
- 5 Sandal
- 6 Shirt
- 7 Sneaker
- 8 Bag
- 9 Ankle boot

**Acknowledgements**

Original dataset was downloaded from TensorFlow datasets catalog.

**License**

The MIT License (MIT) Copyright © [2017] Zalando SE, https://tech.zalando.com

Permission is hereby granted, free of charge, to any person obtaining a copy of this software and associated documentation files (the “Software”), to deal in the Software without restriction, including without limitation the rights to use, copy, modify, merge, publish, distribute, sublicense, and/or sell copies of the Software, and to permit persons to whom the Software is furnished to do so, subject to the following conditions:

The above copyright notice and this permission notice shall be included in all copies or substantial portions of the Software.

THE SOFTWARE IS PROVIDED “AS IS”, WITHOUT WARRANTY OF ANY KIND, EXPRESS OR IMPLIED, INCLUDING BUT NOT LIMITED TO THE WARRANTIES OF MERCHANTABILITY, FITNESS FOR A PARTICULAR PURPOSE AND NONINFRINGEMENT. IN NO EVENT SHALL THE AUTHORS OR COPYRIGHT HOLDERS BE LIABLE FOR ANY CLAIM, DAMAGES OR OTHER LIABILITY, WHETHER IN AN ACTION OF CONTRACT, TORT OR OTHERWISE, ARISING FROM, OUT OF OR IN CONNECTION WITH THE SOFTWARE OR THE USE OR OTHER DEALINGS IN THE SOFTWARE.

## Libraries and Dataset Import

This section contains the necessary libraries (such as tensorflow or pandas) you need to import to run the notebook.

In [ ]:
# general libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

# tensorflow libraries
from tensorflow.keras.datasets import fashion_mnist, mnist
import tensorflow as tf
from tensorflow.keras import layers, losses, models

The following cells are needed to **download** the dataset.

In [ ]:
((trainX, trainY), (testX, testY)) = fashion_mnist.load_data()
labels = ['T-shirt/top', 'Trouser', 'Pullover', 'Dress', 'Coat',
          'Sandal', 'Shirt', 'Sneaker', 'Bag', 'Ankle boot']

**Let's have a look at our data**:

In [ ]:
print('Dimensions of the training dataset: ', trainX.shape)
print('Dimensions of the test dataset: ', testX.shape)
print('Dimensions of the training labels: ', trainY.shape)
print('Dimensions of the test labels: ', testY.shape)

## Dataset Preparation

We now one-hot encode the labels and change the images dimensions, to get easy to use data for later. To know more about one-hot encoding process see the [Further Readings](#fr) section or refer to the hands-on chapter of the book about feed-forward neural networks.

In [ ]:
labels_train = np.zeros((60000, 10))
labels_train[np.arange(60000), trainY] = 1

data_train = trainX.reshape(60000, 28, 28, 1)

In [ ]:
labels_test = np.zeros((10000, 10))
labels_test[np.arange(10000), testY] = 1

data_test = testX.reshape(10000, 28, 28, 1)

In [ ]:
print('Dimensions of the training dataset: ', data_train.shape)
print('Dimensions of the test dataset: ', data_test.shape)
print('Dimensions of the training labels: ', labels_train.shape)
print('Dimensions of the test labels: ', labels_test.shape)

## Data Inspection

In [ ]:
plt.figure(figsize=(12, 3))
rand_ids = np.random.permutation(len(data_train))[:5]
for i, rand_id in enumerate(rand_ids):
  plt.subplot(1, len(rand_ids), i+1)
  plt.title(labels[trainY[rand_id]])
  plt.imshow(data_train[rand_id], cmap='gray')
  plt.gca().set_axis_off()

plt.show()

## Data Normalization

Let's normalize the training data dividing by 255.0 to get the values between 0 and 1.

In [ ]:
data_train_norm = np.array(data_train/255.0)
data_test_norm = np.array(data_test/255.0)

## Convolutional Neural Network Model

Let us try to build one such a network to give you a feeling of how the process would work and how the code looks like. We will not do any hyperparameter tuning or optimization to keep the section understandable.

Our CNN will be made of the following layers:
- **CONV1**: 6 filters 5 x 5, stride $s=1$
- We then apply **ReLU** to the output of the previous layer
- **POOL1** with a window 2 × 2, stride $s=2$
- **CONV2**: 16 Filters 5 × 5, stride $s=1$
- We then apply **ReLU** to the output of the previous layer
- **POOL2** with a window 2 × 2, stride $s=2$
- **Fully Connected Layer** with 128 neurons with activation function ReLU
- **Fully Connected Layer** with 10 neurons for classification of the Zalando dataset
- **Softmax** output neuron

Softmax function here is needed since we one-hot encoded the labels. To have more details about this, see the [Further Readings](#fr) section of the notebook or refer to the hands-on chapter of the book about feed-forward neural networks.


### Defining the Model Architecture

In [ ]:
def build_model():
  # create model
  model = models.Sequential()
  model.add(layers.Conv2D(6, (5, 5), strides = (1, 1), activation = 'relu', input_shape = (28, 28, 1)))
  model.add(layers.MaxPooling2D(pool_size = (2, 2), strides = (2, 2)))
  model.add(layers.Conv2D(16, (5, 5), strides = (1, 1), activation = 'relu'))
  model.add(layers.MaxPooling2D(pool_size = (2, 2), strides = (2, 2)))
  model.add(layers.Flatten())
  model.add(layers.Dense(128, activation = 'relu'))
  model.add(layers.Dense(10, activation = 'softmax'))
  # compile model
  model.compile(loss = 'categorical_crossentropy', optimizer = 'adam', metrics = ['categorical_accuracy'])
  return model

In [ ]:
model = build_model()

In [ ]:
model.summary()

We now train the model.

### Training the Model

In [ ]:
EPOCHS = 10

history = model.fit(
  data_train_norm, labels_train,
  epochs = EPOCHS, verbose = 1,
  batch_size = 100)

And we used the trained model on the test dataset.

### Quantitative and Qualitative Evaluation of the Model

In [ ]:
test_loss, test_accuracy = model.evaluate(data_test_norm, labels_test, verbose = 0)
print('The accuracy on the test set is equal to: ', int(test_accuracy*100), '%.')

In [ ]:
np.random.seed(1)
rand_ids = np.random.permutation(len(data_test))[:5]

plt.figure(figsize=(12, 3))
for i, rand_id in enumerate(rand_ids):
  actual_label = labels[testY[rand_id]]
  predicted_label = np.expand_dims(data_test_norm[rand_id], axis=0)
  predicted_label = labels[np.argmax(model.predict(predicted_label, verbose=False))]

  plt.subplot(1, len(rand_ids), i+1)
  plt.title(f"Predicted: {predicted_label}\nActual: {actual_label}")
  plt.imshow(data_test[rand_id], cmap='gray')
  plt.gca().set_axis_off()

plt.show()

## How robust is the CNN?

Exercise: Pick an image that gets correctly classified by the CNN, then try to change the image such that it gets misclassified.

In [ ]:
def corrupt_image(image):
  # FILL IN HERE
  return image

sample_id = 0
actual_label = labels[testY[sample_id]]

# Predict label for sample image
sample_image = data_test_norm[sample_id]
predicted_label = np.expand_dims(sample_image, axis=0)
predicted_label = labels[np.argmax(model.predict(predicted_label, verbose=False))]
assert predicted_label == actual_label, "Pick a sample that is correctly classified"

# Predict label for corrupted image
corrupted_image = corrupt_image(sample_image)
corrupted_label = np.expand_dims(corrupted_image, axis=0)
corrupted_label = labels[np.argmax(model.predict(corrupted_label, verbose=False))]

plt.figure(figsize=(5, 3))

plt.subplot(1, 2, 1)
plt.title(predicted_label)
plt.imshow(sample_image, cmap='gray', vmin=0, vmax=1)
plt.gca().set_axis_off()

plt.subplot(1, 2, 2)
plt.title(corrupted_label)
plt.imshow(corrupted_image, cmap='gray', vmin=0, vmax=1)
plt.gca().set_axis_off()
plt.show()

## What Filters were learned?

Exercise: Get an impression

- of the filters (convolutional kernels) that were learned during training, and
- of how the input image is transformed by the CNN.

In [ ]:
sample_image = data_test_norm[0]

plt.figure(figsize=(3, 3))
plt.title('Input image')
plt.imshow(sample_image, cmap='gray', vmin=0, vmax=1)
plt.gca().set_axis_off()
plt.show()

model_up_to_first_conv = models.Sequential(model.layers[:1])
model_up_to_second_conv = models.Sequential(model.layers[:3])

for title, submodel in [('After first convolutional layer', model_up_to_first_conv),
                        ('After second convolutional layer', model_up_to_second_conv)]:
  features = submodel.predict(np.expand_dims(sample_image, axis=0), verbose=False)[0]
  features = np.split(features, features.shape[2], axis=2)

  print()
  n_cols = 4
  n_rows = int(np.ceil(len(features) / n_cols))
  plt.figure(figsize=(8, n_rows * 2))
  for i, feature in enumerate(features):
    plt.subplot(n_rows, n_cols, i+1)
    plt.imshow(feature, cmap='gray')
    if i == 0: plt.title(title)
    plt.gca().set_axis_off()

  plt.show()

## Generating Adversarial Examples

Here, we try to fool the convolutional neural network into misclassifying an image. Instead of using backpropagation to compute how to change the weights to reduce the loss, here we use backpropagation to compute **how to change the image to increase the loss**.

Specifically, we use the "Fast Gradient Sign Method" from:

Goodfellow, Shlens, and Szegedy (2014) arXiv  
Explaining and Harnessing Adversarial Examples  
https://arxiv.org/abs/1412.6572

In [ ]:
def compute_gradient_sign(model, image, label):
  image = tf.convert_to_tensor(np.expand_dims(image, axis=0))
  label = tf.convert_to_tensor(np.expand_dims(label, axis=0))

  loss_function = losses.CategoricalCrossentropy()
  with tf.GradientTape() as tape:
    tape.watch(image)
    prediction = model(image)
    loss = loss_function(label, prediction)

  gradient = tape.gradient(loss, image)[0]
  grad_sign = np.sign(gradient)
  return grad_sign

In [ ]:
sample_id = 1
sample_image = data_test_norm[sample_id]
sample_label = labels_test[sample_id]

# Compute the sign of the gradient of the loss with respect to the image
grad_sign = compute_gradient_sign(model, sample_image, sample_label)

# Perturb the original input image
adversarial_perturbation = 0.1 * grad_sign
adversarial_sample = sample_image + adversarial_perturbation

plt.figure(figsize=(9, 3))

for i, (image, title) in enumerate([
    (sample_image, 'Original image'),
    (adversarial_perturbation, 'Adversarial perturbation'),
    (adversarial_sample, 'Adversarial sample')]):
  prob = model.predict(np.expand_dims(image, axis=0), verbose=False)[0]
  label = labels[np.argmax(prob)]
  prob = np.max(prob)

  plt.subplot(1, 3, i+1)
  plt.imshow(image, cmap='gray', vmin=0, vmax=1)
  plt.title(f"{title}\n{int(100 * prob):d}% {label}")
  plt.gca().set_axis_off()

plt.show()

## Fine-tuning the Fashion Model on MNIST Digits

In practice, you likely need much more complex convolutional neural networks (CNN) with many more parameters. At the same time, it is challenging to generate a labeled dataset with millions of samples.

A common strategy in this situation is to "fine-tune" a larger CNN. This means: reusing the convolutional layers (sometimes called the "backbone") for feature extraction, but training a new, final fully-connected layer (sometimes called "readout layer") specifically for the problem at hand. Because the readout layer contains only few parameters compared to the entire model, you can get away with little training data.

**Exercise:** Fine-tune the Fashion MNIST model on the original MNIST dataset that consists of hand-written digits. Try to find out how many training samples are needed to satisfactory performance of the fine-tuned model.

**HINT:** If you have $N$ training samples, a batch size of $B$, and train for $E$ epochs, then the model parameters will be optimized over $N / B \times E$ gradient steps. Thus, when reducing the number of training samples, you might have to increase the number of epochs for model parameters to reach an optimum.

### Loading the MNIST Dataset

In [ ]:
(mnist_x_train, mnist_y_train), (mnist_x_test, mnist_y_test) = mnist.load_data()

In [ ]:
# Preparing MNIST training data
mnist_x_train = mnist_x_train.reshape(60000, 28, 28, 1)
mnist_x_train = np.array(mnist_x_train / 255.0)

mnist_labels_train = np.zeros((60000, 10))
mnist_labels_train[np.arange(60000), mnist_y_train] = 1

# Preparing MNIST test data
mnist_x_test = mnist_x_test.reshape(10000, 28, 28, 1)
mnist_x_test = np.array(mnist_x_test / 255.0)

mnist_labels_test = np.zeros((10000, 10))
mnist_labels_test[np.arange(10000), mnist_y_test] = 1

In [ ]:
plt.figure(figsize=(12, 3))
rand_ids = np.random.permutation(len(mnist_x_train))[:5]
for i, rand_id in enumerate(rand_ids):
  plt.subplot(1, len(rand_ids), i+1)
  plt.imshow(mnist_x_train[rand_id], cmap='gray', vmin=0, vmax=1)
  plt.gca().set_axis_off()

plt.show()

In [ ]:
# Reuse convolutional layers from Fashion model
mnist_model = models.Sequential(model.layers[:-1])

# Do not (re)train convolutions from Fashion model
for layer in mnist_model.layers:
  layer.trainable = False

# Add new dense ("readout") layers for MNIST model
mnist_model.add(layers.Dense(10, activation = 'softmax'))
mnist_model.compile(loss = 'categorical_crossentropy', optimizer = 'adam', metrics = ['categorical_accuracy'])
mnist_model.summary()

In [ ]:
EPOCHS = 10
n_training_samples = 10_000

np.random.seed(0)
training_sample_ids = np.random.permutation(len(mnist_x_train))
training_sample_ids = training_sample_ids[:n_training_samples]

mnist_model.fit(
  mnist_x_train[training_sample_ids],
  mnist_labels_train[training_sample_ids],
  epochs = EPOCHS,
  batch_size = 100)

In [ ]:
_, mnist_test_accuracy = mnist_model.evaluate(mnist_x_test, mnist_labels_test, verbose = 0)
print('The accuracy on the test set is equal to: ', int(mnist_test_accuracy*100), '%.')

## Exercises

1. [*Easy Difficulty*] Try to build a multiclass classification model like the one we saw together in this notebook, but with a different dataset, the MNIST database of handwritten digits (http://yann.lecun.com/exdb/mnist/). To download the dataset from TensorFlow use the following lines of code:
```
from tensorflow import keras
(x_train, y_train), (x_test, y_test) = keras.datasets.mnist.load_data()
```
2. [*Medium Difficulty*] Try to change the network's parameters to see if you can get a higher accuracy. Change kernel size, stride, and padding.

## Further Readings

**Fashion-MNIST dataset**

1. Xiao, Han, Kashif Rasul, and Roland Vollgraf. "Fashion-mnist: a novel image dataset for benchmarking machine learning algorithms." arXiv preprint arXiv:1708.07747 (2017)

**One-hot encoding, integer encoding, softmax function**

1. https://deepai.org/machine-learning-glossary-and-terms/one-hot-encoding (what one-hot encoding means)
2. https://towardsdatascience.com/softmax-activation-function-how-it-actually-works-d292d335bd78 (why we need to use softmax function and the difference between one-hot encoding and integer encoding)